In [1]:
from transformers import AutoTokenizer , AutoConfig , AutoModelForSeq2SeqLM

In [2]:
model_name = "t5-small"
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name , use_fast = True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name , config = config)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from datasets import load_dataset
dataset = load_dataset('findnitai/english-to-hinglish')

  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
master = []
for line in dataset['train']['translation']:
    master.append(line['en'])
    master.append(line['hi_ng'])

def gen_training_data():
    return (master[i : i+500]
    for i in range(0, len(master), 500)
    )
tokenizer_training_data = gen_training_data()

In [6]:
tokenizer_training_data

<generator object gen_training_data.<locals>.<genexpr> at 0x7f8079eeb920>

In [7]:
trained_tokenizer = tokenizer.train_new_from_iterator(tokenizer_training_data , vocab_size = 32128)

In [8]:
trained_tokenizer.save_pretrained("english_to_hinglish_tokenizer")

('english_to_hinglish_tokenizer/tokenizer_config.json',
 'english_to_hinglish_tokenizer/special_tokens_map.json',
 'english_to_hinglish_tokenizer/tokenizer.json')

In [21]:
source_prefix = "Translate English to Hinglish : "
source_lang = "en"
target_lang = "hi_ng"
max_source_length = 128 
max_target_length = 128 # target and source length task dependent (translation, summary etc.)
padding = "max_length" # padding to max length
num_epochs = 3

In [9]:
dataset_dict = dataset.data

In [10]:
output_json_path = "english_to_hinglish.json"

In [11]:
formatted_dataset = [{'translation':{"en": example["en"], "hi_ng": example["hi_ng"]}} for example in dataset['train']['translation']]

In [12]:
import json
with open(output_json_path, 'w', encoding='utf-8') as jsonl_file:
    for item in formatted_dataset :
        jsonl_file.write(json.dumps(item) + "\n")

In [13]:
raw_datasets = load_dataset(
    "json",
    data_files="english_to_hinglish.json",
  )

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-a9faf385ec4e7638/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
def preprocess(source_data):
    inputs = [k[source_lang] for k in source_data['translation']]
    targets = [k[target_lang] for k in source_data['translation']]
    inputs = [source_prefix + inp for inp in inputs]
    #using the previously trained tokenizer
    model_inputs = trained_tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)
    labels = trained_tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

    labels["input_id"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [16]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 189102
    })
})

In [19]:
train_dataset = raw_datasets["train"]

In [22]:
train_dataset = train_dataset.map(preprocess, batched=True, remove_columns="translation")

  0%|          | 0/190 [00:00<?, ?ba/s]

In [23]:
from transformers import default_data_collator

In [24]:
data_collator = default_data_collator

trainer_args_in = {
    'output_dir': 'full-hinglish-translator',
    'overwrite_output_dir' : True,
    'do_train' : True,
    # 'do_valid' : False,
    'per_device_train_batch_size' : 8,
    'num_train_epochs' : num_epochs,
}

In [25]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, HfArgumentParser
parser = HfArgumentParser((Seq2SeqTrainingArguments))
training_args = parser.parse_dict(trainer_args_in)

trainer = Seq2SeqTrainer(model=model, args=training_args[0], train_dataset=train_dataset, tokenizer=trained_tokenizer, data_collator=data_collator)

train_result = trainer.train(resume_from_checkpoint=None)
trainer.save_model()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.829900
1000,0.494900
1500,0.451000
2000,0.429400
2500,0.399500
3000,0.395500
3500,0.386200
4000,0.369100
4500,0.350600
5000,0.350900


RuntimeError: [enforce fail at inline_container.cc:337] . unexpected pos 200064704 vs 200064592

--- Logging error ---
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 1104, in emit
    self.flush()
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 1084, in flush
    self.stream.flush()
OSError: [Errno 28] No space left on device
Call stack:
  File "/opt/conda/lib/python3.10/threading.py", line 973, in _bootstrap
    self._bootstrap_inner()
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/internal/internal_util.py", line 49, in run
    self._run()
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/internal/internal_util.py", line 100, in _run
    self._process(record)
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/internal/internal.py", line 279, in _process
    self._hm.handle(record)
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/internal/handler.py", line 136, in handle
    handler(record)


In [ ]:
pip show transformers